In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [2]:
import pandas as pd
import numpy as np

In [3]:
# path = "/content/gdrive/MyDrive/dataset/"
path = '../dataset new/'


In [4]:
df_users_full = pd.read_csv(path + "users.csv")
df_users1 = df_users_full[["user_id","functional_unit","department"]]
df_users = df_users1[df_users1.functional_unit == "2 - ResearchAndEngineering"]
df_users = df_users[(df_users.department != "1 - Research")]
df_users = pd.DataFrame(df_users)

In [5]:
df_users = df_users.dropna(axis = 0)
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3991 entries, 8 to 17333
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          3991 non-null   object
 1   functional_unit  3991 non-null   object
 2   department       3991 non-null   object
dtypes: object(3)
memory usage: 124.7+ KB


In [6]:
df_device = pd.read_csv(path + "device.csv")

In [7]:
df_file = pd.read_csv(path + "file.csv")

In [8]:
df_logon = pd.read_csv(path + "logon.csv")

In [9]:
df_psychometric = pd.read_csv(path + "psychometric.csv")
df_psychometric = df_psychometric[['employee_name','user_id','O','C','E', 'A', 'N']]

In [10]:
df_users_clean = df_users.rename(columns= {'user_id':'user'}, inplace=False)

In [11]:
df_logon_users = pd.merge(df_logon, df_users_clean, on = 'user')
df_logon_users_clean = df_logon_users.drop(columns=['functional_unit', 'department'])


In [12]:
df_device_users = pd.merge(df_device, df_users_clean, on='user')
df_device_users_clean = df_device_users.drop(columns=['functional_unit','department'])

In [13]:
df_device_users = pd.merge(df_device, df_users_clean, on = 'user')
df_device_users_clean = df_device_users.drop(columns = ['functional_unit', 'department'])


In [14]:
df_file_users = pd.merge(df_file, df_users_clean, on = 'user')
df_file_users_clean = df_file_users.drop(columns = ['functional_unit', 'department'])


In [15]:
df_psychometric_users = pd.merge(df_psychometric, df_users, on = 'user_id')
df_psychometric_users_clean = df_psychometric_users[['employee_name', 'user_id','O', 'C', 'E','A','N']]


In [16]:
df_logon_users_clean['date'] = pd.to_datetime(df_logon_users_clean['date'])

In [17]:
df_logon_users_clean['time'] = df_logon_users_clean['date'].dt.time

In [18]:
df_user_logon = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logon']

In [19]:
df_user_logon['hour'] = pd.to_datetime(df_user_logon['date'], format='%H:%M').dt.hour

/tmp/ipykernel_21504/1930962928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logon['hour'] = pd.to_datetime(df_user_logon['date'], format='%H:%M').dt.hour


In [20]:
df_user_logon_stats = df_user_logon.groupby('user')['time'].agg([min,max]).reset_index()

In [21]:
df_logon_mode = df_user_logon.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [22]:
df_logon_mean = df_user_logon.groupby('user')['hour'].mean().reset_index()
df_logon_mean['hour'].dtype
df_logon_mean['hour'] = pd.to_datetime(df_logon_mean['hour'], format='%H').dt.time


In [23]:
df_user_logon_stats['mode'] = df_logon_mode['time']
df_user_logon_stats['mean'] = df_logon_mean['hour']

In [24]:
df_user_logoff = df_logon_users_clean.loc[df_logon_users_clean['activity'] == 'Logoff']

In [25]:
df_user_logoff['date'] = pd.to_datetime(df_user_logoff['date'])
df_user_logoff['time'] = df_user_logoff['date'].dt.time

/tmp/ipykernel_21504/636462469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['date'] = pd.to_datetime(df_user_logoff['date'])
/tmp/ipykernel_21504/636462469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['time'] = df_user_logoff['date'].dt.time


In [26]:

df_user_logoff['hour'] = pd.to_datetime(df_user_logoff['date'], format='%H:%M').dt.hour

/tmp/ipykernel_21504/1504136962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_logoff['hour'] = pd.to_datetime(df_user_logoff['date'], format='%H:%M').dt.hour


In [27]:
df_user_logoff_stats = df_user_logoff.groupby('user')['time'].agg([min,max]).reset_index()

In [28]:
df_user_logoff.groupby('user')['time'].agg(pd.Series.mode).reset_index()

,user,time
0,AAN0077,16:56:00
1,AAW0103,16:14:00
2,ACE0265,16:06:00
3,ACL0394,17:15:00
4,ADR0362,16:45:00
...,...,...
225,XTM0246,17:00:00
226,YJT0368,16:45:00
227,YSB0779,16:45:00
228,ZBL0379,19:30:00


In [29]:
df_logoff_mode = df_user_logoff.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [30]:
df_logoff_mean = df_user_logoff.groupby('user')['hour'].mean().reset_index()
df_logoff_mean['hour'] = df_logoff_mean['hour'].astype(int)
df_logoff_mean['hour'] = pd.to_datetime(df_logoff_mean['hour'], format='%H').dt.time


In [31]:
df_user_logoff_stats['mode'] = df_logoff_mode['time']
df_user_logoff_stats['mean'] = df_logoff_mean['hour']

In [32]:
df_device_users_clean['time'] = pd.to_datetime(df_device_users_clean['date']).dt.time

In [33]:
df_device_conn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Connect']
df_device_disconn = df_device_users_clean.loc[df_device_users_clean['activity'] == 'Disconnect']


In [34]:
df_device_conn_stats = df_device_conn.groupby('user')['time'].agg([min, max]).reset_index()

In [35]:
df_device_conn_stats_1 = df_device_conn.groupby('user')

In [36]:
df_conn_mode = df_device_conn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()

In [37]:
df_device_conn['hour'] = pd.to_datetime(df_device_conn['date']).dt.hour

/tmp/ipykernel_21504/2737355688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_device_conn['hour'] = pd.to_datetime(df_device_conn['date']).dt.hour


In [38]:
df_conn_mean = df_device_conn.groupby('user')['hour'].mean().reset_index()
df_conn_mean['hour'] = df_conn_mean['hour'].astype(int)
df_conn_mean['hour'] = pd.to_datetime(df_conn_mean['hour'],format="%H").dt.time

In [39]:
df_device_conn_stats['mode'] = df_conn_mode['time']
df_device_conn_stats['mean'] = df_conn_mean['hour']

In [40]:
df_device_disconn_stats = df_device_disconn.groupby('user')['time'].agg([min,max]).reset_index()

In [41]:
df_dconn_mode = df_device_disconn.groupby('user')['time'].agg(lambda x: x.value_counts().index[0]).reset_index()



In [42]:
df_device_disconn['hour'] = pd.to_datetime(df_device_disconn['date']).dt.hour


/tmp/ipykernel_21504/1753353138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_device_disconn['hour'] = pd.to_datetime(df_device_disconn['date']).dt.hour


In [43]:
df_dconn_mean = df_device_disconn.groupby('user')['hour'].mean().reset_index()
df_dconn_mean['hour'] = df_dconn_mean['hour'].astype(int)
df_dconn_mean['hour'] = pd.to_datetime(df_dconn_mean['hour'], format='%H').dt.time


In [44]:
df_dconn_mean

,user,hour
0,AJQ0376,14:00:00
1,AJR0231,11:00:00
2,AOD0066,12:00:00
3,ARH0777,12:00:00
4,BCP0247,12:00:00
5,BDS0396,13:00:00
6,BMS0057,15:00:00
7,BZK0095,11:00:00
8,CAE0080,12:00:00
9,CGH0088,12:00:00


In [45]:
df_device_disconn_stats['mode'] = df_dconn_mode['time']
df_device_disconn_stats['mean'] = df_dconn_mean['hour']

In [46]:
df_file_users_clean['date2'] = pd.to_datetime(df_file_users_clean['date']).dt.date

In [47]:
df_files_per_day = df_file_users_clean.groupby(['user', 'date2']).size().reset_index()

In [48]:
df_files_per_day.rename(columns={0:'transfers_per_day'}, inplace=True)


In [49]:
df_files_max_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(max).reset_index()
df_files_max_per_day.rename(columns={'transfers_per_day': "max_transfers_per_user"}, inplace=True)

In [50]:
df_files_mode_per_day = df_files_per_day.groupby('user')['transfers_per_day'].agg(lambda x: x.value_counts().index[0]).reset_index()
df_files_mode_per_day.rename(columns={'transfers_per_day': "mode_transfers_per_user"}, inplace=True)


In [51]:
df_files_mode_per_day

,user,mode_transfers_per_user
0,AJQ0376,486
1,AJR0231,90
2,AOD0066,30
3,ARH0777,36
4,BCP0247,54
5,BDS0396,72
6,BMS0057,36
7,BZK0095,486
8,CAE0080,36
9,CGH0088,18


In [52]:
df_files_stats = df_files_mode_per_day

In [53]:
df_files_stats_new = pd.DataFrame()
df_files_stats_new['user'] = df_files_stats['user']
df_files_stats_new['mode_trasfers_per_user'] = df_files_stats['mode_transfers_per_user']
df_files_stats_new['max_transfers_per_user'] = df_files_max_per_day['max_transfers_per_user']


In [54]:
df_user_pc = df_logon_users_clean.groupby(['user','pc',]).agg(pc_visits_per_user_total = pd.NamedAgg(column = 'pc', aggfunc = 'count')).reset_index()
df_user_pc['count'] = df_user_pc.groupby(['user'])['pc'].transform('nunique')
df_user_pc = df_user_pc.drop(['pc', 'pc_visits_per_user_total'], axis=1)
df_user_pc = df_user_pc.drop_duplicates()

In [55]:
df_user_pc

,user,count
0,AAN0077,1
1,AAW0103,1
2,ACE0265,1
3,ACL0394,2
5,ADR0362,1
...,...,...
632,XTM0246,1
633,YJT0368,2
635,YSB0779,1
636,ZBL0379,1


In [56]:
# import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [57]:
df_user_pc_count = np.array(df_user_pc['count'])

In [58]:
df_user_pc_ct = df_user_pc_count.reshape(-1,1)

In [59]:
import pickle
model = pickle.load(open('../pkl/user_pc_ct.pkl', 'rb'))
graph_a_score = model.decision_function(df_user_pc_ct)
threat = model.predict(df_user_pc_ct)


In [60]:
#user pc
#forest = IsolationForest(bootstrap=False, contamination=0.1,max_features=1.0,
#                          max_samples='auto',n_estimators=100, n_jobs=1, random_state=None,
#                          verbose=0)
# # forest.fit(df_user_pc_ct)


In [61]:
# import pickle 
# with open('user_pc_ct.pkl','wb') as file :
#   pickle.dump(forest, file)

In [62]:
# graph_a_score = forest.decision_function(df_user_pc_ct)
# print(graph_a_score[1:10])

In [63]:
graph_result = pd.DataFrame()
graph_result['user'] = df_user_pc['user']
graph_result['ascore'] = graph_a_score
graph_result['threat']= threat
graph_result.to_csv('../output/user_pc_ct.csv', index=False)
graph_result


,user,ascore,threat
0,AAN0077,0.090545,1
1,AAW0103,0.090545,1
2,ACE0265,0.090545,1
3,ACL0394,0.000000,1
5,ADR0362,0.090545,1
...,...,...,...
632,XTM0246,0.090545,1
633,YJT0368,0.000000,1
635,YSB0779,0.090545,1
636,ZBL0379,0.090545,1


In [64]:
outliers = graph_result.loc[graph_result['ascore'] < 0]
print(outliers)

        user    ascore  threat
14   AJR0231 -0.242110      -1
54   ALC0100 -0.169568      -1
96   ARH0777 -0.168848      -1
110  BGZ0902 -0.255862      -1
154  CGH0088 -0.168848      -1
167  CQS0899 -0.222694      -1
205  DNS0397 -0.224169      -1
218  EER0383 -0.181953      -1
254  FED0275 -0.263095      -1
401  LWB0078 -0.356151      -1
489  PYT0264 -0.204651      -1
580  WJP0386 -0.266216      -1
592  WMP0272 -0.190906      -1


In [65]:
def dtt2timestamp(dtt):
  time_in_sec = (dtt.hour*60 + dtt.minute) * 60 + dtt.second
  return time_in_sec

In [66]:
df_user_logon_stats_sec = df_user_logon_stats


In [67]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['min']]
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logon_stats_sec['mean']]

In [68]:
df_user_logon_stats_sec['min_ts'] = min_ts
df_user_logon_stats_sec['max_ts'] = max_ts
df_user_logon_stats_sec['mode_ts'] = mode_ts
df_user_logon_stats_sec['mean_ts'] = mean_ts

In [69]:
df_user_logon_stats_sec.drop(['min','max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,27900,29580,29580,25200
1,AAW0103,25200,26280,26100,25200
2,ACE0265,28800,30300,29700,28800
3,ACL0394,32400,60757,33300,32400
4,ADR0362,27000,28080,28080,25200
...,...,...,...,...,...
225,XTM0246,27900,29520,29460,25200
226,YJT0368,27000,68459,27900,25200
227,YSB0779,27000,28020,27900,25200
228,ZBL0379,29700,52585,30600,36000


In [70]:
df_user_logoff_stats_sec = df_user_logoff_stats


In [71]:
min_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['min']] 
max_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['max']]
mode_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mode']]
mean_ts = [dtt2timestamp(dtt) for dtt in df_user_logoff_stats_sec['mean']]

In [72]:
df_user_logoff_stats_sec['min_ts'] = min_ts
df_user_logoff_stats_sec['max_ts'] = max_ts
df_user_logoff_stats_sec['mode_ts'] = mode_ts
df_user_logoff_stats_sec['mean_ts'] = mean_ts

In [73]:
df_user_logoff_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

,user,min_ts,max_ts,mode_ts,mean_ts
0,AAN0077,60420,62100,60960,57600
1,AAW0103,58320,59400,58440,57600
2,ACE0265,57900,59400,57960,57600
3,ACL0394,33235,63000,62100,57600
4,ADR0362,60120,61200,60300,57600
...,...,...,...,...,...
225,XTM0246,60480,62100,61200,57600
226,YJT0368,27293,68732,60300,54000
227,YSB0779,60180,61200,60300,57600
228,ZBL0379,69420,71100,70200,68400


In [74]:
df_log_on_off_stats = pd.DataFrame()

df_log_on_off_stats['user'] = df_user_logon_stats_sec['user']
df_log_on_off_stats['on_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['on_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['on_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['on_mean_ts'] = df_user_logon_stats_sec['mean_ts']
df_log_on_off_stats['off_min_ts'] = df_user_logon_stats_sec['min_ts']
df_log_on_off_stats['off_max_ts'] = df_user_logon_stats_sec['max_ts']
df_log_on_off_stats['off_mode_ts'] = df_user_logon_stats_sec['mode_ts']
df_log_on_off_stats['off_mean_ts'] = df_user_logon_stats_sec['mean_ts']

In [75]:
df_log_on_off_stats.dtypes

user           object
on_min_ts       int64
on_max_ts       int64
on_mode_ts      int64
on_mean_ts      int64
off_min_ts      int64
off_max_ts      int64
off_mode_ts     int64
off_mean_ts     int64
dtype: object

In [76]:
log_stats = df_log_on_off_stats.drop(['user'], axis = 1)
log_stats_matrix = np.matrix(log_stats)
print(log_stats_matrix)

[[27900 29580 29580 ... 29580 29580 25200]
 [25200 26280 26100 ... 26280 26100 25200]
 [28800 30300 29700 ... 30300 29700 28800]
 ...
 [27000 28020 27900 ... 28020 27900 25200]
 [29700 52585 30600 ... 52585 30600 36000]
 [25200 26760 26100 ... 26760 26100 25200]]


In [77]:
df_log_on_off_stats.columns

Index(['user', 'on_min_ts', 'on_max_ts', 'on_mode_ts', 'on_mean_ts',
       'off_min_ts', 'off_max_ts', 'off_mode_ts', 'off_mean_ts'],
      dtype='object')

In [78]:
# 

model2 = pickle.load(open('../pkl/log_stats_matrix.pkl', 'rb'))
log_ascore = model2.decision_function(log_stats_matrix)
threat = model2.predict(log_stats_matrix)


In [79]:
# # counting the values
# df22 = pd.Series(forest.predict(log_stats_matrix))
# # df22 = df22.map({1:0, -1:1})
# # print(df22.value_counts())

In [80]:
# import pickle 
# with open('log_stats_matrix.pkl','wb') as file :
#   pickle.dump(forest, file)

In [81]:
df_user_log_result = pd.DataFrame()
df_user_log_result['user'] = df_user_logoff_stats_sec['user']
df_user_log_result['ascore'] = log_ascore
df_user_log_result['threat'] = threat
df_user_log_result.to_csv('../output/user_log_result.csv', index=False)
print(df_user_log_result)


        user    ascore  threat
0    AAN0077  0.039010       1
1    AAW0103  0.106512       1
2    ACE0265  0.153949       1
3    ACL0394  0.033959       1
4    ADR0362  0.147203       1
..       ...       ...     ...
225  XTM0246  0.060684       1
226  YJT0368  0.038474       1
227  YSB0779  0.192625       1
228  ZBL0379  0.111829       1
229  ZIF0269  0.092330       1

[230 rows x 3 columns]


In [82]:
df_user_log_result.loc[df_user_log_result['ascore'] < 0]

,user,ascore,threat
29,BKA0340,-0.026654,-1
31,BMS0057,-0.011392,-1
43,BZV0067,-0.015218,-1
56,CQS0899,-0.119682,-1
65,DJS0406,-0.030006,-1
73,EER0383,-0.028553,-1
77,FED0275,-0.103707,-1
83,GFM0250,-0.045716,-1
86,GJP0098,-0.023272,-1
95,HML0060,-0.041845,-1


In [83]:
df_device_conn_stats_sec = df_device_conn_stats
con_min_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['min']]
con_max_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['max']]
con_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mode']]
con_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_conn_stats_sec['mean']]

df_device_conn_stats_sec['min_ts'] = con_min_ts
df_device_conn_stats_sec['max_ts'] = con_max_ts
df_device_conn_stats_sec['mode_ts'] = con_mode_ts
df_device_conn_stats_sec['mean_ts'] = con_mean_ts
df_device_conn_stats_sec = df_device_conn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)

In [84]:
df_device_disconn_stats_sec = df_device_conn_stats
discon_min_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['min']]
discon_max_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['max']]
discon_mode_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mode']]
discon_mean_ts = [dtt2timestamp(dtt) for dtt in df_device_disconn_stats_sec['mean']]

df_device_disconn_stats_sec['min_ts'] = discon_min_ts
df_device_disconn_stats_sec['max_ts'] = discon_max_ts
df_device_disconn_stats_sec['mode_ts'] = discon_mode_ts
df_device_disconn_stats_sec['mean_ts'] = discon_mean_ts
df_device_disconn_stats_sec = df_device_disconn_stats_sec.drop(['min', 'max','mode','mean'], axis=1)


In [85]:
df_device_full = pd.DataFrame()
df_device_full['user'] = df_device_conn_stats['user']

In [86]:
df_device_full['con_min_ts'] = df_device_conn_stats_sec['min_ts']
df_device_full['con_max_ts'] = df_device_conn_stats_sec['max_ts']
df_device_full['con_mode_ts'] = df_device_conn_stats_sec['mode_ts']
df_device_full['con_mean_ts'] = df_device_conn_stats_sec['mean_ts']

# disconnect stats
df_device_full['dcon_min_ts'] = df_device_disconn_stats_sec['min_ts']
df_device_full['dcon_max_ts'] = df_device_disconn_stats_sec['max_ts']
df_device_full['dcon_mode_ts'] = df_device_disconn_stats_sec['mode_ts']
df_device_full['dcon_mean_ts'] = df_device_disconn_stats_sec['mean_ts']

# files per day stats
df_device_full['file_mode'] = df_files_stats_new['mode_trasfers_per_user']
df_device_full['file_max'] = df_files_stats_new['max_transfers_per_user']

In [87]:
device_full_matrix = df_device_full.drop(['user'],axis=1)
device_params = np.matrix(device_full_matrix)
device_params[:10]

matrix([[30401, 68694, 43193, 43200, 30401, 68694, 43193, 43200,   486,
           486],
        [27100, 56918, 29523, 36000, 27100, 56918, 29523, 36000,    90,
           252],
        [27693, 55895, 28247, 36000, 27693, 55895, 28247, 36000,    30,
            72],
        [27756, 59689, 41949, 39600, 27756, 59689, 41949, 39600,    36,
           144],
        [ 4870, 84764, 52773, 39600,  4870, 84764, 52773, 39600,    54,
           198],
        [33007, 61926, 35672, 39600, 33007, 61926, 35672, 39600,    72,
           234],
        [ 1185, 76882, 57035, 43200,  1185, 76882, 57035, 43200,    36,
           234],
        [28882, 59022, 45455, 39600, 28882, 59022, 45455, 39600,   486,
           486],
        [16018, 83814, 33156, 39600, 16018, 83814, 33156, 39600,    36,
           198],
        [29154, 57723, 32112, 39600, 29154, 57723, 32112, 39600,    18,
           198]])

In [88]:
df_device_full.columns

Index(['user', 'con_min_ts', 'con_max_ts', 'con_mode_ts', 'con_mean_ts',
       'dcon_min_ts', 'dcon_max_ts', 'dcon_mode_ts', 'dcon_mean_ts',
       'file_mode', 'file_max'],
      dtype='object')

In [89]:
# # device (-user)
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)

# forest.fit(device_params)


In [90]:
# import pickle 
# with open('device_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [91]:
# 

model3 = pickle.load(open('../pkl/device_params.pkl', 'rb'))
log_ascore = model3.decision_function(device_params)
threat = model3.predict(device_params)


In [92]:
df_device_file_full_result = pd.DataFrame()

df_device_file_full_result['user'] = df_device_full['user']
df_device_file_full_result['ascore'] = log_ascore
df_device_file_full_result['threat'] = threat

df_device_file_full_result.to_csv('../output/device_file_full_result.csv', index=False)

In [93]:
df_device_file_full_result.loc[df_device_file_full_result['ascore'] < 0] 

,user,ascore,threat
6,BMS0057,-0.055519,-1
8,CAE0080,-0.010819,-1
12,CSD0242,-0.068050,-1
29,LJW0776,-0.050707,-1
35,MSB0359,-0.031661,-1
37,QLC0248,-0.084706,-1
40,SBM0063,-0.111420,-1
46,ZBL0379,-0.000021,-1


In [94]:
psychometric_matrix = df_psychometric_users_clean.drop(['user_id', 'employee_name'], axis = 1)
psychometric_params = np.matrix(psychometric_matrix)

In [95]:
# # psycho ds 
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(psychometric_params)

In [96]:
# import pickle 
# with open('psychometric_params.pkl','wb') as file :
#   pickle.dump(forest, file)

In [98]:
# 

model4 = pickle.load(open('../pkl/psychometric_params.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model4.decision_function(psychometric_params)
threat = model4.predict(psychometric_params)


In [99]:
df_psychometric_result = pd.DataFrame()

df_psychometric_result['user'] = df_psychometric_users_clean['user_id']
df_psychometric_result['ascore'] = log_ascore
df_psychometric_result['threat']= threat
df_psychometric_result.to_csv('../output/psychometric_result.csv', index=False)
df_psychometric_result

,user,ascore,threat
0,HBB0090,0.034275,1
1,HBB0090,0.034275,1
2,HBB0090,0.034275,1
3,HBB0090,0.034275,1
4,HBB0090,0.034275,1
...,...,...,...
3986,GGK0375,0.006880,1
3987,GGK0375,0.006880,1
3988,GGK0375,0.006880,1
3989,GGK0375,0.006880,1


In [100]:
df_psychometric_result.loc[df_psychometric_result['ascore'] < 0]

,user,ascore,threat
203,JAB0249,-0.020013,-1
204,JAB0249,-0.020013,-1
205,JAB0249,-0.020013,-1
206,JAB0249,-0.020013,-1
207,JAB0249,-0.020013,-1
...,...,...,...
3968,QKA0388,-0.014503,-1
3969,QKA0388,-0.014503,-1
3970,QKA0388,-0.014503,-1
3971,QKA0388,-0.014503,-1


In [101]:
df = pd.merge(df_log_on_off_stats, df_user_pc, on='user')

In [102]:
df_1 = pd.merge(df, df_psychometric, left_on = 'user', right_on = 'user_id')

In [103]:
df_1.head()

,user,on_min_ts,on_max_ts,on_mode_ts,on_mean_ts,off_min_ts,off_max_ts,off_mode_ts,off_mean_ts,count,employee_name,user_id,O,C,E,A,N
0,AAN0077,27900,29580,29580,25200,27900,29580,29580,25200,1,Allistair Akeem Nichols,AAN0077,42,15,32,22,29
1,AAW0103,25200,26280,26100,25200,25200,26280,26100,25200,1,Alexa Allegra Walton,AAW0103,39,42,47,39,28
2,ACE0265,28800,30300,29700,28800,28800,30300,29700,28800,1,Amos Carson Emerson,ACE0265,26,23,14,19,40
3,ACL0394,32400,60757,33300,32400,32400,60757,33300,32400,2,Adam Clark Lindsey,ACL0394,25,44,18,19,17
4,ADR0362,27000,28080,28080,25200,27000,28080,28080,25200,1,Amir Damian Rosario,ADR0362,32,42,35,38,26


In [104]:
df_final = df_1.drop(['employee_name', 'user_id'], axis=1)


In [105]:
df_all_parameters = df_final

In [106]:
df_all_parameters_input = df_all_parameters.drop(['user'], axis = 1)

In [107]:
print(df_all_parameters_input)

     on_min_ts  on_max_ts  on_mode_ts  on_mean_ts  off_min_ts  off_max_ts  \
0        27900      29580       29580       25200       27900       29580   
1        25200      26280       26100       25200       25200       26280   
2        28800      30300       29700       28800       28800       30300   
3        32400      60757       33300       32400       32400       60757   
4        27000      28080       28080       25200       27000       28080   
..         ...        ...         ...         ...         ...         ...   
225      27900      29520       29460       25200       27900       29520   
226      27000      68459       27900       25200       27000       68459   
227      27000      28020       27900       25200       27000       28020   
228      29700      52585       30600       36000       29700       52585   
229      25200      26760       26100       25200       25200       26760   

     off_mode_ts  off_mean_ts  count   O   C   E   A   N  
0          29580

In [108]:
# # psycho logon ds
# forest = IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
#         max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
#         verbose=0)
# forest.fit(df_all_parameters_input)

In [109]:
# import pickle 
# with open('all_parameters_input.pkl','wb') as file :
#   pickle.dump(forest, file)

In [111]:
# 

model5 = pickle.load(open('../pkl/all_parameters_input.pkl', 'rb'))
# df2=pd.DataFrame()
log_ascore = model5.decision_function(df_all_parameters_input)
threat = model5.predict(df_all_parameters_input)


In [112]:
df_all_parameters_result = pd.DataFrame()

df_all_parameters_result['user'] = df_final['user']
df_all_parameters_result['ascore'] =log_ascore
df_all_parameters_result['threat'] = threat
print(df_all_parameters_result)
df_all_parameters_result.to_csv('../output/all_parameters_result.csv', index=False)

        user    ascore  threat
0    AAN0077  0.083207       1
1    AAW0103  0.027089       1
2    ACE0265  0.082833       1
3    ACL0394 -0.012597      -1
4    ADR0362  0.102599       1
..       ...       ...     ...
225  XTM0246  0.078112       1
226  YJT0368  0.026272       1
227  YSB0779  0.085627       1
228  ZBL0379  0.026624       1
229  ZIF0269  0.049846       1

[230 rows x 3 columns]
